In [24]:
import pandas as pd
import datetime as dt
import locale
import holidays

In [25]:
# --- Configurar los días festivos para Colombia para 2025:
co_holidays = holidays.CountryHoliday('CO', years=[2025])

In [26]:
ruta ="/home/donsson/proyectos/BIOMETRICO/dataset/Biometrico BQ 10.2025.xls"
df = pd.read_excel(ruta)
mes = 10
sucursal =  "BQ"

# Definir fechas (Mantenemos el rango de Octubre 2025 para el ejemplo)
FECHA_INICIO = '2025-10-01'
FECHA_FIN = '2025-10-27'

WARNING *** file size (72409) not 512 + multiple of sector size (512)


In [27]:
# Manejar por buenas practicas titulos en minuscula y sin espacios en blanco
new_name =[] 
for column in df.columns:
    minus = column.lower()
    replace = minus.replace(" ","_")
    new_name.append(replace)

df.columns = new_name
df.duplicated().sum()


np.int64(0)

In [28]:
#cambia el nombre de la columna de numero a ID del trabajador
df.rename(columns={'número': 'id'}, inplace=True) 

In [29]:
# Reemplaza el espacio no-breaking por un espacio normal
df['tiempo'] = df['tiempo'].str.replace('\xa0', ' ', regex=False)  

# Reemplaza a.m a AM para que datetime lo lea 
df['tiempo'] = df['tiempo'].str.replace('a. m.', 'AM', regex=False, case=False) 

# Reemplaza p.m a PM para que datetime lo lea 
df['tiempo'] = df['tiempo'].str.replace('p. m.', 'PM', regex=False, case=False) 


In [30]:
df[df["nombre"]=="SAID  VARGAS"]

,id,nombre,tiempo,estado,dispositivos,tipo_de_registro
0,100223197,SAID VARGAS,24/10/2025 1:42:04 PM,NaN,barranquilla,0
1,100223197,SAID VARGAS,24/10/2025 2:42:56 PM,NaN,barranquilla,0
2,100223197,SAID VARGAS,24/10/2025 5:32:50 PM,Salida,barranquilla,0


In [31]:
df['tiempo'] = pd.to_datetime(df['tiempo'], dayfirst=True, errors='coerce')

# Crear columnas adicionales
df['fecha'] = df['tiempo'].dt.date
df['hora'] = df['tiempo'].dt.time
df['mes'] = df['tiempo'].dt.month


/tmp/ipykernel_19191/2676730200.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['tiempo'] = pd.to_datetime(df['tiempo'], dayfirst=True, errors='coerce')


In [32]:
# Extraer la hora (en formato 24 horas)
df['hora_entera'] = df['tiempo'].dt.hour

# Extraer los minutos
df['minutos_enteros'] = df['tiempo'].dt.minute

# Calcular la fracción de la hora (como en Excel)
df['fraccion_dia_hora'] = (df['hora_entera'] + df['minutos_enteros'] / 60) / 24


In [33]:
# Crear una lista de nombres de días de la semana en español para mapear
dias_semana_espanol = ["lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo"]

# Extraer el número del día de la semana (0=lunes, 6=domingo)
#    Lo aplicamos a la parte de la fecha de la columna 'tiempo'
df['dia_Numero'] = df['tiempo'].dt.weekday

# Mapear el número a su nombre correspondiente en español
df['dia'] = df['dia_Numero'].map(lambda x: dias_semana_espanol[x])


# Busqueda de dias festivos
df['festivo'] = df['fecha'].apply(lambda x: x in co_holidays)


df = df[df["mes"]==mes]


fecha_inicio = df["fecha"].min()
fecha_fin = df["fecha"].max()


df["nombre"] = df["nombre"].str.upper()



empleados = {
    "ANDRES FELIPE ORDOÑEZ GREGORY": "mostrador",
    "CARLOS ANDRES CABYALES PRADO": "mostrador",
    "CLAUDIA XIMENA ORTIZ CAMACHO": "mostrador",
    "DAVID DE JESUS FERNANDEZ TETTE": "mostrador",
    "DIEGO ANDRES CARDONA VILLA": "mostrador",
    "EDINSON JAVIER GARCIA MURILLO": "mostrador",
    "EDISON MIGUEL CIFUENTES OSORIO": "mostrador",
    "EDWIN FERNANDO COLLAZOS BARRETO": "mostrador",
    "EVERTH DE JESUS OQUENDO JIMENEZ": "externo",
    "GERMAN ANTONIO MORALES CORDENAS": "mostrador",
    "JAIME CAÑON": "externo",
    "JAIME ANDRES CABALLERO CABALLERO": "mostrador",
    "JAIME ANDRES DUARTE MORA": "externo",
    "JAIME SEBASTIAN RODRIGUEZ RAMOS": "mostrador",
    "JEYSSON FABIAN BLANCO RODRIGUEZ": "mostrador",
    "JHON MULATO": "mostrador",
    "JUAN DAVID SOGAMOSO TIRADO": "mostrador",
    "JUAN JOSE OSPINA CARDONA": "mostrador",
    "KENER DAVID SANTAMARIA": "mostrador",
    "MARCO MAURICIO MEJIA RAMIREZ": "mostrador",
    "MARIA CAMILA ARGUELLO BERMUDEZ": "mostrador",
    "MIGUEL NIEBLES": "externo",
    "NATALI ESPINAL TRUJILLO": "mostrador",
    "NEYDER CRESPO": "mostrador",
    "OMAR YADIR HERNANDEZ PULGARIN": "externo",
    "PATRICIA NUÑEZ VALENCIA": "externo",
    "SAID  VARGAS": "mostrador",
    "SANDRA PATICIA RAMIREZ CORREDOR": "mostrador",
    "SANDRA ROCIO FONSECA MOYANO": "mostrador",
    "VICTOR ALEJANDRO GONZALEZ RODRIGUEZ": "mostrador",
    "WILSON ALEJANDRO PEÑAEZ PEREZ": "mostrador",
    "YADI ALMEETH RODRIGUEZ JOTA": "mostrador",
    "YERITZA MARIA PALMA BARRERA": "mostrador",
    "YURI NATALIA OJANA PARRA": "mostrador",
    "AURELIYASTRA SERI FAAS ROCHA": "mostrador",
    "JUAN ESTEBAN TORRES ROCHA": "mostrador",
    "YONEINKER JOSE GONZALEZ HERNANDEZ": "mostrador",
    "YEIS MUÑOZ": "externo"
}

df["cargo"] = df["nombre"].map(empleados)
df["nombre"].unique()

array(['SAID  VARGAS', 'NEYDER CRESPO', 'MIGUEL NIEBLES',
       'KENER DAVID SANTAMARIA', 'DAVID DE JESUS FERNANDEZ TETTE'],
      dtype=object)

In [34]:
df.head(10)

,id,nombre,tiempo,estado,dispositivos,tipo_de_registro,fecha,hora,mes,hora_entera,minutos_enteros,fraccion_dia_hora,dia_Numero,dia,festivo,cargo
0,100223197,SAID VARGAS,2025-10-24 13:42:04,NaN,barranquilla,0,2025-10-24,13:42:04,10,13,42,0.570833,4,viernes,False,mostrador
1,100223197,SAID VARGAS,2025-10-24 14:42:56,NaN,barranquilla,0,2025-10-24,14:42:56,10,14,42,0.612500,4,viernes,False,mostrador
2,100223197,SAID VARGAS,2025-10-24 17:32:50,Salida,barranquilla,0,2025-10-24,17:32:50,10,17,32,0.730556,4,viernes,False,mostrador
3,101009066,NEYDER CRESPO,2025-10-01 08:29:34,Entrada,barranquilla,0,2025-10-01,08:29:34,10,8,29,0.353472,2,miércoles,False,mostrador
4,101009066,NEYDER CRESPO,2025-10-01 13:36:42,NaN,barranquilla,0,2025-10-01,13:36:42,10,13,36,0.566667,2,miércoles,False,mostrador
5,101009066,NEYDER CRESPO,2025-10-01 14:47:40,NaN,barranquilla,0,2025-10-01,14:47:40,10,14,47,0.615972,2,miércoles,False,mostrador
6,101009066,NEYDER CRESPO,2025-10-01 17:34:21,Salida,barranquilla,0,2025-10-01,17:34:21,10,17,34,0.731944,2,miércoles,False,mostrador
7,101009066,NEYDER CRESPO,2025-10-02 08:44:42,Entrada,barranquilla,0,2025-10-02,08:44:42,10,8,44,0.363889,3,jueves,False,mostrador
8,101009066,NEYDER CRESPO,2025-10-02 13:31:44,NaN,barranquilla,0,2025-10-02,13:31:44,10,13,31,0.563194,3,jueves,False,mostrador
9,101009066,NEYDER CRESPO,2025-10-02 14:44:39,NaN,barranquilla,0,2025-10-02,14:44:39,10,14,44,0.613889,3,jueves,False,mostrador


In [35]:
# 2. Renombrar 'tiempo' a 'timestamp' para que la lógica de agrupación funcione
df.rename(columns={'tiempo': 'timestamp'}, inplace=True) 

# 3. Crear columna 'fecha' a partir del timestamp (necesaria para el filtro)
df['fecha'] = df['timestamp'].dt.date
df_filtrado = df[df['timestamp'].dt.date <= pd.to_datetime(FECHA_FIN).date()].copy()


# 3. Cálculo de horas trabajadas (Tu lógica)
horarios = df_filtrado.groupby(['nombre', 'fecha', 'cargo'])['timestamp'].agg(['min', 'max']).reset_index()
horarios.rename(columns={'min': 'hora_entrada', 'max': 'hora_salida'}, inplace=True)
horarios['horas_trabajadas'] = (horarios['hora_salida'] - horarios['hora_entrada']).dt.total_seconds() / 3600
horarios['mes'] = pd.to_datetime(horarios['fecha']).dt.month # Convertir de nuevo a datetime para extraer mes


resumen = horarios.groupby(['nombre','cargo' ,'mes'])['horas_trabajadas'].sum().reset_index()
resumen.rename(columns={'horas_trabajadas': 'total_horas_efectivas'}, inplace=True)

resumen.head()

,nombre,cargo,mes,total_horas_efectivas
0,DAVID DE JESUS FERNANDEZ TETTE,mostrador,10,158.643333
1,KENER DAVID SANTAMARIA,mostrador,10,183.635278
2,MIGUEL NIEBLES,externo,10,81.165278
3,NEYDER CRESPO,mostrador,10,175.459167
4,SAID VARGAS,mostrador,10,3.846111


In [36]:
import pandas as pd
import holidays

def calcular_horas_esperadas(fecha_inicio, fecha_fin, tipo_empleado='Mostrador', country_code='CO'):
    """
    Calcula las horas laborales esperadas entre dos fechas, excluyendo domingos y feriados.
    Maneja diferentes horarios según el tipo de empleado.
    """
    
    # Normalizar tipo de empleado
    tipo_empleado_lower = tipo_empleado.lower().strip()
    
    # Definir horarios según tipo de empleado
    if tipo_empleado_lower == 'mostrador':
        # Mostrador: Lunes a viernes 9 horas, Sábado 4 horas
        horas_lunes_viernes = 9
        horas_sabado = 4
        usar_horario_externo = False
    elif tipo_empleado_lower == 'externo':
        # Vendedor Externo:
        # Lunes: 10 horas (7:00 a.m. - 5:00 p.m.)
        # Martes a viernes: 9 horas (8:00 a.m. - 5:00 p.m.)
        # Sábado: 3 horas (8:00 a.m. - 11:00 a.m.)
        horas_lunes = 10
        horas_martes_viernes = 9
        horas_sabado = 3
        usar_horario_externo = True
    else:
        # Por defecto usar horario de Mostrador
        horas_lunes_viernes = 9
        horas_sabado = 4
        usar_horario_externo = False

    total_horas = 0.0

    # Crear un objeto de la librería holidays para Colombia
    dias_festivos = holidays.CountryHoliday(country_code)

    # Generar un rango de fechas
    fechas = pd.to_datetime(pd.Series(pd.date_range(start=fecha_inicio, end=fecha_fin)))

    for fecha in fechas:
        # Excluir domingos
        if fecha.dayofweek == 6:  # Domingo = 6
            continue

        # Excluir feriados
        if fecha.date() in dias_festivos: # Usar fecha.date() para comparar con el set de holidays
            continue

        # Sumar horas según el tipo de empleado y día de la semana
        if usar_horario_externo:
            if fecha.dayofweek == 0:  # Lunes
                total_horas += horas_lunes
            elif fecha.dayofweek >= 1 and fecha.dayofweek <= 4:  # Martes a viernes
                total_horas += horas_martes_viernes
            elif fecha.dayofweek == 5:  # Sábado
                total_horas += horas_sabado
        else:  # Mostrador
            if fecha.dayofweek >= 0 and fecha.dayofweek <= 4:  # Lunes a viernes
                total_horas += horas_lunes_viernes
            elif fecha.dayofweek == 5:  # Sábado
                total_horas += horas_sabado

    return total_horas


# ==============================================================================
# NUEVA FUNCIÓN AÑADIDA: Para aplicar el cálculo a un DataFrame de empleados
# ==============================================================================

def calcular_horas_para_dataframe(df, fecha_inicio, fecha_fin, columna_cargo='cargo', country_code='CO'):
    """
    Calcula las horas esperadas para cada empleado en un DataFrame según su cargo.
    Agrega la columna 'Horas_Esperadas' al DataFrame original.
    
    Args:
        df (DataFrame): DataFrame con información de empleados.
        fecha_inicio (str): Fecha de inicio en formato 'YYYY-MM-DD'.
        fecha_fin (str): Fecha de fin en formato 'YYYY-MM-DD'.
        columna_cargo (str): Nombre de la columna que contiene el cargo.
        country_code (str): Código del país para los feriados.
    
    Returns:
        DataFrame: DataFrame original con columna adicional 'Horas_Esperadas'.
    """
    
    def determinar_tipo_empleado(cargo):
        """Determina el tipo de empleado basándose en el cargo."""
        cargo_str = str(cargo).lower().strip()
        
        # Buscar si contiene "externo" en el cargo
        if 'externo' in cargo_str:
            return 'Externo'
        # Buscar si contiene "mostrador" en el cargo
        elif 'mostrador' in cargo_str:
            return 'Mostrador'
        else:
            # Por defecto, si no coincide con ninguno, usar Mostrador
            return 'Mostrador'
    
    # Calcular horas para cada empleado
    horas_esperadas = []
    
    # Usar .apply para eficiencia, aunque el for loop también es válido
    for cargo in df[columna_cargo]:
        tipo_empleado = determinar_tipo_empleado(cargo)
        # Llamar a la función de cálculo de horas con el tipo de empleado determinado
        horas = calcular_horas_esperadas(fecha_inicio, fecha_fin, tipo_empleado, country_code)
        horas_esperadas.append(horas)
    
    # Agregar columna al DataFrame original
    df['Horas_Esperadas'] = horas_esperadas
    
    return df




In [37]:
resumen_df_con_horas = calcular_horas_para_dataframe(
    resumen, 
    fecha_inicio=FECHA_INICIO, 
    fecha_fin=FECHA_FIN,
    columna_cargo='cargo' 
)

In [38]:
resumen_df_con_horas.head(10)

,nombre,cargo,mes,total_horas_efectivas,Horas_Esperadas
0,DAVID DE JESUS FERNANDEZ TETTE,mostrador,10,158.643333,178.0
1,KENER DAVID SANTAMARIA,mostrador,10,183.635278,178.0
2,MIGUEL NIEBLES,externo,10,81.165278,177.0
3,NEYDER CRESPO,mostrador,10,175.459167,178.0
4,SAID VARGAS,mostrador,10,3.846111,178.0


In [39]:
resumen = resumen_df_con_horas.copy()
resumen['porcentaje_cumplimiento'] = resumen['total_horas_efectivas'] / resumen['Horas_Esperadas']

In [40]:
import pandas as pd
import numpy as np

    


def calcular_horas_esperadas_por_dia(row):
    # 1. Obtener el día de la semana (0=Lunes, 6=Domingo)
    dia_semana = row["fecha"].weekday()

    # 2. Normalizar cargo
    cargo_lower = str(row["cargo"]).lower().strip()

        # 3. Reglas
    if "externo" in cargo_lower:
        if dia_semana == 0:   # Lunes
            return 10
        elif dia_semana == 5: # Sábado
            return 3
        else:                 # Martes-Viernes
            return 9
    elif "mostrador" in cargo_lower:
        if dia_semana == 5:#Sabado
            return 4
        else:
            return 9


        
# --- 3. Aplicar función para calcular horas esperadas ---
horarios["fecha"] = pd.to_datetime(horarios["fecha"])
horarios["horas_esperadas"] = horarios.apply(calcular_horas_esperadas_por_dia, axis=1)


    
 # --- 4. Detectar errores ---
horarios["trabajo_menos"] = horarios["horas_trabajadas"].round(2) < horarios["horas_esperadas"]



In [41]:
horarios = horarios.drop(columns=["mes"])

In [42]:
horarios.sample(10)

,nombre,fecha,cargo,hora_entrada,hora_salida,horas_trabajadas,horas_esperadas,trabajo_menos
62,NEYDER CRESPO,2025-10-07,mostrador,2025-10-07 08:23:37,2025-10-07 18:26:26,10.046944,9,False
59,NEYDER CRESPO,2025-10-02,mostrador,2025-10-02 08:44:42,2025-10-02 17:35:48,8.851667,9,True
71,NEYDER CRESPO,2025-10-20,mostrador,2025-10-20 08:34:46,2025-10-20 17:32:09,8.956389,9,True
65,NEYDER CRESPO,2025-10-10,mostrador,2025-10-10 06:58:25,2025-10-10 16:04:52,9.107500,9,False
47,MIGUEL NIEBLES,2025-10-07,externo,2025-10-07 08:25:36,2025-10-07 08:25:36,0.000000,9,True
42,KENER DAVID SANTAMARIA,2025-10-27,mostrador,2025-10-27 08:36:18,2025-10-27 17:24:49,8.808611,9,True
45,MIGUEL NIEBLES,2025-10-03,externo,2025-10-03 09:23:54,2025-10-03 16:44:54,7.350000,9,True
15,DAVID DE JESUS FERNANDEZ TETTE,2025-10-20,mostrador,2025-10-20 07:22:40,2025-10-20 16:19:04,8.940000,9,True
57,MIGUEL NIEBLES,2025-10-27,externo,2025-10-27 07:05:04,2025-10-27 17:00:27,9.923056,10,True
52,MIGUEL NIEBLES,2025-10-20,externo,2025-10-20 12:01:37,2025-10-20 17:04:54,5.054722,10,True


In [43]:
with pd.ExcelWriter(f"entregas/reporte_{mes}_{sucursal}.xlsx") as writer:
    resumen.to_excel(writer , sheet_name ="resumen",index=False)
    horarios.to_excel(writer , sheet_name= "diario", index =False)
print("Archivo creado correctamente")

Archivo creado correctamente
